Parte 1: Generación del conjunto de datos

In [ ]:
import pandas as pd
import numpy as np
import unicodedata
import re
import os
import csv
import sklearn.metrics as metrics

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 30.5 MB/s 
     |████████████████████████████████| 3.3 MB 64.1 MB/s 
     |████████████████████████████████| 895 kB 68.9 MB/s 
     |████████████████████████████████| 67 kB 7.2 MB/s 
     |████████████████████████████████| 596 kB 62.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
tsv_file = open("lcp_single_test.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")

In [ ]:
label = 1
count = 0
data_raw = pd.DataFrame(columns = range(5))
for row in read_tsv:
  if label == 1:
    data_raw.columns = row
    label = 0
  else:
    data_raw.loc[count] = [row[0], row[1], row[2], row[3], row[4]]
    count += 1


In [ ]:
data_raw.head(10)

,id,corpus,sentence,token,complexity
0,3K8CQCU3KE19US5SN890DFPK3SANWR,bible,"But he, beckoning to them with his hand to be ...",hand,0.0
1,3Q2T3FD0ON86LCI41NJYV3PN0BW3MV,bible,"If I forget you, Jerusalem, let my right hand ...",hand,0.19736842105263158
2,3ULIZ0H1VA5C32JJMKOTQ8Z4GUS51B,bible,"the ten sons of Haman the son of Hammedatha, t...",hand,0.2
3,3BFF0DJK8XCEIOT30ZLBPPSRMZQTSD,bible,Let your hand be lifted up above your adversar...,hand,0.2678571428571429
4,3QREJ3J433XSBS8QMHAICCR0BQ1LKR,bible,"Abimelech chased him, and he fled before him, ...",entrance,0.0
5,30F94FBDNRKF12C1PDZGZ0IRI9XTB1,bible,"for as he thinks about the cost, so he is.",cost,0.25
6,3JHB4BPSFK9JLT5CAPRP3PCEA2CQ9R,bible,"When his armor bearer saw that Saul was dead, ...",armor,0.02777777777777779
7,3Y40HMYLL1I1EIURUEH8TTVLKQ9XUM,bible,"Therefore put on the whole armor of God, that ...",armor,0.2
8,30Y6N4AHYPWV0KXTZKLRUB59XBPRDA,bible,"Hezekiah listened to them, and showed them all...",armor,0.2222222222222222
9,38O9DZ0A62NFBR1TA7YCNXN94XN26F,bible,"That first slaughter, which Jonathan and his a...",armor,0.3194444444444444


In [ ]:
data = pd.DataFrame(columns=['text', 'complexity'])
count = 0
for i in range(0, data_raw.shape[0]):
  data.loc[count] = [data_raw['sentence'][count].lower() + ' [SEP] ' + data_raw['token'][count].lower(), \
                    data_raw['complexity'][count]]
  count += 1


In [ ]:
data['text'][0]

'but he, beckoning to them with his hand to be silent, declared to them how the lord had brought him out of the prison. [SEP] hand'

In [ ]:
# Preprocess, tokenize and vectorize texts and labels
def vectorize_text(s, max_length):

    # Unicode normalization
    #s = s.strip().lower()
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

    # Remove undesired characters
    #s = re.sub(r'([.¿?()\'",;:$€])', r' \1 ', s)
    #s = re.sub(r"[^a-zA-Záéíóú.,!?;:()$€]+", r" ", s)

    # Text to tensor
    input_ids = tokenizer.encode(
      s,
      add_special_tokens=True,
      max_length=max_length,
      padding='longest', 
      truncation=True,
      return_tensors='np'
    )
    return input_ids[0]

In [ ]:
data['text_vec'] = data.apply(lambda r: vectorize_text(r['text'], 512), axis=1)

In [ ]:
data

,text,complexity,text_vec
0,"but he, beckoning to them with his hand to be ...",0.0,"[101, 2021, 2002, 1010, 10272, 13369, 2000, 20..."
1,"if i forget you, jerusalem, let my right hand ...",0.19736842105263158,"[101, 2065, 1045, 5293, 2017, 1010, 6744, 1010..."
2,"the ten sons of haman the son of hammedatha, t...",0.2,"[101, 1996, 2702, 4124, 1997, 10654, 2319, 199..."
3,let your hand be lifted up above your adversar...,0.2678571428571429,"[101, 2292, 2115, 2192, 2022, 4196, 2039, 2682..."
4,"abimelech chased him, and he fled before him, ...",0.0,"[101, 11113, 14428, 2571, 2818, 13303, 2032, 1..."
...,...,...,...
882,"the report by mr philippe busquin, on behalf o...",0.5694444444444444,"[101, 1996, 3189, 2011, 2720, 11169, 3902, 125..."
883,section v - court of auditors (sec(2002) 405 -...,0.5357142857142857,"[101, 2930, 1058, 1011, 2457, 1997, 20964, 201..."
884,"- mr president, on 1 june, on the basis of inf...",0.546875,"[101, 1011, 2720, 2343, 1010, 2006, 1015, 2238..."
885,i would like to extend a warm welcome to this ...,0.5714285714285714,"[101, 1045, 2052, 2066, 2000, 7949, 1037, 4010..."


In [ ]:
# Separar los conjuntos de datos
data = data.sample(frac=1)
train_portion = 0.9
split_point = int(train_portion*len(data))
train_data, test_data =  data[:split_point], data[split_point:] 
print(len(train_data), 'train, ', len(test_data), 'test')

798 train,  89 test


In [ ]:
#Create dataset en pytorch
from torch.utils.data import Dataset, DataLoader
import torch

class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        
    def __getitem__(self, index):
        input_ids = torch.tensor(self.data.text_vec.iloc[index]).cpu()
        attention_mask = torch.ones([input_ids.size(0)]).cpu()
        targets = self.data.complexity.iloc[index]
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': targets
         }
            
    def __len__(self):
        return self.len

In [ ]:
train_set, test_set = MyDataset(train_data), MyDataset(test_data)

In [ ]:
test_set.data

,text,complexity,text_vec
349,"one might then ask: ""how then could the role o...",0.3088235294117647,"[101, 2028, 2453, 2059, 3198, 1024, 1000, 2129..."
18,"the wolf and the lamb shall feed together, and...",0.20588235294117646,"[101, 1996, 4702, 1998, 1996, 12559, 4618, 543..."
347,while they do have their limitations (e.g. dev...,0.21666666666666667,"[101, 2096, 2027, 2079, 2031, 2037, 12546, 100..."
762,out of concern about developments relating to ...,0.26190476190476186,"[101, 2041, 1997, 5142, 2055, 8973, 8800, 2000..."
77,and one tenth part for every lamb of the seven...,0.1607142857142857,"[101, 1998, 2028, 7891, 2112, 2005, 2296, 1255..."
...,...,...,...
187,"or a charmer, or a consulter with a familiar s...",0.3157894736842105,"[101, 2030, 1037, 11084, 2121, 1010, 2030, 103..."
67,"when this sound was heard, the multitude came ...",0.3088235294117647,"[101, 2043, 2023, 2614, 2001, 2657, 1010, 1996..."
652,the directive ensures rights to all individual...,0.25,"[101, 1996, 16449, 21312, 2916, 2000, 2035, 36..."
411,reversal learning in rodents can serve as an i...,0.26136363636363635,"[101, 23163, 4083, 1999, 28156, 2064, 3710, 20..."


In [ ]:
train_set.data

,text,complexity,text_vec
757,our initial contacts have provided a sound bas...,0.296875,"[101, 2256, 3988, 10402, 2031, 3024, 1037, 261..."
482,well-formed plaques remained in the treated an...,0.4,"[101, 2092, 1011, 2719, 28487, 2815, 1999, 199..."
210,"also we inform you, that touching any of the p...",0.36363636363636365,"[101, 2036, 2057, 12367, 2017, 1010, 2008, 724..."
56,"you shall stone him to death with stones, beca...",0.23529411764705882,"[101, 2017, 4618, 2962, 2032, 2000, 2331, 2007..."
278,we did not find differences in food intake or ...,0.20588235294117646,"[101, 2057, 2106, 2025, 2424, 5966, 1999, 2833..."
...,...,...,...
615,we shall thoroughly examine the issue regardin...,0.20238095238095238,"[101, 2057, 4618, 12246, 11628, 1996, 3277, 49..."
656,"the next item is the report by iles braghetto,...",0.31666666666666665,"[101, 1996, 2279, 8875, 2003, 1996, 3189, 2011..."
760,the charging of heavy goods vehicles for the u...,0.41666666666666663,"[101, 1996, 13003, 1997, 3082, 5350, 4683, 200..."
405,it is possible that a subtle increase in m-ops...,0.33999999999999997,"[101, 2009, 2003, 2825, 2008, 1037, 11259, 362..."


In [ ]:
len(test_set)

89

In [ ]:
len(train_set)
#len(test_set)

798

In [ ]:
from transformers import BertForSequenceClassification, BertConfig
config = BertConfig.from_pretrained('bert-large-uncased')
config.problem_type='regression'
config.num_labels=1
model = BertForSequenceClassification.from_pretrained('bert-large-uncased', config=config)


Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [ ]:
from transformers import Trainer, TrainingArguments

def collate_batch(batch):
    """ Optimize memory by setting all vectors in batch to a length equal to max
        length found
    """
    
    def pad_sequence(in_tensor, max_size):
        """ Fill tensor with zeros up to max_size
        """
        out_tensor = np.zeros(max_size)
        out_tensor[:in_tensor.size(0)] = in_tensor.numpy()
        return out_tensor
    
    batch_inputs = []
    batch_attention_masks = []
    batch_targets = []

    max_size = max([ex['input_ids'].size(0) for ex in batch])
    for item in batch:
        batch_inputs.append(pad_sequence(item['input_ids'], max_size))
        batch_attention_masks.append(pad_sequence(item['attention_mask'], max_size))
        batch_targets.append(float(item['labels']))
    return {
        "input_ids": torch.tensor(batch_inputs, dtype=torch.long),
        "attention_mask": torch.tensor(batch_attention_masks, dtype=torch.long),
        "labels": torch.tensor(batch_targets, dtype=torch.float)
    }

class MyTrainer(Trainer):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)

  def get_train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        collate_fn=collate_batch
    )

  def get_eval_dataloader(self, eval_dataset):
    return DataLoader(
        self.eval_dataset,
        collate_fn=collate_batch
    )


In [ ]:
training_args = TrainingArguments(
    output_dir='output',
    evaluation_strategy='steps',
    eval_steps=100,
    num_train_epochs=10,
    remove_unused_columns=False,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
)

trainer = MyTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 798
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 7980


Step,Training Loss,Validation Loss
100,No log,0.017827
200,No log,0.165766
300,No log,0.049686
400,No log,0.016951
500,0.076900,0.076750
600,0.076900,0.048277
700,0.076900,0.016840


***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
Saving model checkpoint to output/checkpoint-500
Configuration saved in output/checkpoint-500/config.json
Model weights saved in output/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
***** Running Evaluation *****
  Num examples = 89
  Batch size = 1
